##Imports, Downloading the data and cleaning
We use the dataset from huggingface


In [2]:
#Requirements for colab
#!pip install transformers hackernews-client datasets -q

     |████████████████████████████████| 3.5 MB 29.0 MB/s 
     |████████████████████████████████| 312 kB 67.1 MB/s 
     |████████████████████████████████| 6.8 MB 45.5 MB/s 
     |████████████████████████████████| 596 kB 51.8 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 895 kB 70.9 MB/s 
     |████████████████████████████████| 243 kB 71.8 MB/s 
     |████████████████████████████████| 133 kB 78.0 MB/s 
     |████████████████████████████████| 1.1 MB 61.9 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 77.1 MB/s 
     |████████████████████████████████| 271 kB 47.4 MB/s 


In [3]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from datasets import Dataset
import numpy as np
from datasets import load_metric
SEED = 42

In [48]:
dataset = load_dataset("Fraser/news-category-dataset")
num_labels = len(set(dataset["train"]["category_num"]))

Using custom data configuration default
Reusing dataset news_category (/root/.cache/huggingface/datasets/Fraser___news_category/default/0.0.0/737b7b6dff469cbba49a6202c9e94f9d39da1fed94e13170cf7ac4b61a75fb9c)


  0%|          | 0/3 [00:00<?, ?it/s]

In [76]:
reduced_cats = {
  "CULTURE & ARTS":["ARTS","ARTS & CULTURE","CULTURE & ARTS"],
 "BUSINESS": ["BUSINESS","MONEY"],
 "EDUCATION": ["EDUCATION","COLLEGE"],
 "COMEDY & ENTERTAINMENT" : ["COMEDY","ENTERTAINMENT","MEDIA"],
 "HEALTH & LIVING": ["WELLNESS","HEALTHY LIVING",
                     "STYLE & BEAUTY","HOME & LIVING",
                     "PARENTS","STYLE","FOOD & DRINK","TASTE","PARENTING","DIVORCE","WEDDINGS"],
 "RELIGION" : ["RELIGION"],
 "POLITICS" : ["POLITICS","BLACK VOICES","LATINO VOICES","QUEER VOICES","WOMEN"],
 "SPORTS" : ["SPORTS"],
 "TRAVEL" : ["TRAVEL"],
 "NEWS" :["GOOD NEWS","THE WORLDPOST","WORLDPOST","WORLD NEWS","WEIRD NEWS","CRIME"],
 "ENVIRONMENT" : ["GREEN","ENVIRONMENT"],
 "SCIENCE": ["SCIENCE"],
 "TECH": ["TECH"],
 "OTHER" : ["IMPACT","FIFTY"]
 }


In [77]:
label_map = {}
for i,cats in enumerate(reduced_cats.values()):
  for cat in cats:
    label_map[cat] = i


In [79]:

train_dataset = Dataset.from_dict({
    "text" : dataset["train"]["headline"],
    "label": pd.Series(dataset["train"]["category"]).replace(label_map).tolist()
})

In [80]:

test_dataset = Dataset.from_dict({
    "text" : dataset["test"]["headline"],
    "label": pd.Series(dataset["test"]["category"]).replace(label_map).tolist()
})

## Training the model
Training the model

In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
    
model_checkpoint = "microsoft/xtremedistil-l6-h256-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


#num_labels = num_cat #len(set(dataset["train"]["label"]))
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/525 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
def preprocess_function(examples, tokenizer):
    return tokenizer(examples["text"],
                   padding="max_length",max_length=201 ,
                   truncation=True)



In [81]:
encoded_train_dataset = train_dataset.map(preprocess_function, tokenizer,batched=True)

  0%|          | 0/161 [00:00<?, ?ba/s]

In [82]:
encoded_test_dataset = test_dataset.map(preprocess_function, tokenizer,batched=True)

  0%|          | 0/31 [00:00<?, ?ba/s]

In [83]:
metric_name = "accuracy"
batch_size= 16
args = TrainingArguments(
    f"finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
#    label_names=["category"],
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [84]:
from datasets import load_metric

metric = load_metric("accuracy")


In [85]:
def compute_metrics(eval_pred, metric):
#    print(eval_pred)
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [88]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    
    compute_metrics=compute_metrics(metric)
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 160682
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 50215


Epoch,Training Loss,Validation Loss


## Interference
Checking the model

In [ ]:
model_test = AutoModelForSequenceClassification.from_pretrained(f"./finetuned/checkpoint-{trainer.state.global_step}/",
                                                                num_labels=num_labels)

In [ ]:
#https://github.com/huggingface/transformers/blob/6f0a9b41efc3cb88de33e56eea7e5d5699eb6be0/src/transformers/pipelines/text_classification.py

In [ ]:
#source: https://github.com/huggingface/transformers/blob/6f0a9b41efc3cb88de33e56eea7e5d5699eb6be0/src/transformers/pipelines/text_classification.py
def softmax(_outputs):
    maxes = np.max(_outputs, axis=-1, keepdims=True)
    shifted_exp = np.exp(_outputs - maxes)
    return shifted_exp / shifted_exp.sum(axis=-1, keepdims=True)

def test_interference(trained_model, text, tokenizer):
  text_pt = tokenizer([text],
                    padding="max_length",max_length=201,
                    truncation=True,return_tensors="pt")
  #return np.argmax(model_test(**text_pt)[0][0].detach().numpy())
  return softmax(trained_model(**text_pt)[0][0].detach().numpy())

In [ ]:
test_interference(model_test, "A bird was flying today", tokenizer)

6

In [ ]:
headline_and_comment[23]

'22-year-old builds chips in his parents’ garage I&#x27;m more interested how he managed to master the knowledge of chip manufacturing plus a ton of electronics repairs (see his microscope and other equipment) at such an early age. Very admirable.<p>Look at what he did:<p>&gt; Photolithography machines are expensive—up to $150 million—and so Zeloof made his own by bolting a modified conference room projector bought on Amazon onto a microscope. It projects his designs at tiny scale onto silicon wafers that Zeloof coats in material sensitive to ultraviolet light.<p>&gt; One of Zeloof’s best finds was a broken electron microscope that cost $250,000 in the early ’90s; he bought it for $1,000 and repaired it. He uses it to inspect his chips for flaws, as well as the nanostructures on butterfly wings.<p>Damn this guy is good.'

In [ ]:
test_interference(headline_and_comment[25])

'BUSINESS'

# Fetch Hackernews
Fetch headline and first comment

In [ ]:
from hackernews import hn
news_client = hn.NewsClient()

def get_comm(x, requests):
  try:
    c_id = x.kids[0]
    return requests.get(f"https://hacker-news.firebaseio.com/v0/item/{c_id}.json?print=pretty").json()['text']
  except:
    return ""

h = news_client.get_best_story(fetchMax=200)
import requests
headline_and_comment  =  [x.title + " " + get_comm(x, requests) for x in h]


## Param tuning

In [ ]:
def model_init(AutoModelForSequenceClassification, model_checkpoint, num_labels):
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

trainer = Trainer(
    model_init=model_init(AutoModelForSequenceClassification, model_checkpoint, num_labels),
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")